# Buy/Sell Decision Function

In [3]:
import numpy as np
import pandas as pd
import numpy_financial as npf

In [7]:
def price_it(ticker_list,discountrate, marginrate):
    dfprice = pd.DataFrame(columns =['ticker','annualgrowthrate','last_eps','future_eps'])
    for f in ticker_list:
        agr =  f.eps_growth.mean()*.01 #growth rate
        
        last_eps = f.eps.mean()
        
        years  = 10 #period
        
        future_eps = abs(npf.fv(agr,years,0,last_eps))
        

        dfprice.loc[0] = [f.ticker[0],agr,last_eps,future_eps]
         
        min_eps= f.pe.min()

        dfprice['pe_ratio'] = min_eps

        dfprice['FV'] = dfprice['future_eps']*dfprice['pe_ratio']

        #discountrate=.15 or .20
        #margin=.15

        ticker=f.ticker[0]
        stock_pricing = data_pull(f'{ticker}') #
        stock_price = stock_pricing['Close'] #
        cur_stock_price = stock_price[0]
        
        
        dfprice['PV'] = abs(npf.pv(discountrate,years,0,fv=dfprice['FV']))

        if dfprice['FV'].values[0] > 0:
            dfprice['marginprice'] = dfprice['PV']*(1-marginrate)
        else:
            dfprice['marginprice'] = 0

        dfprice['lastshareprice']=cur_stock_price

        print(dfprice)

        if dfprice['lastshareprice'][0]<dfprice['marginprice'][0]:
            dfprice['decision'] = 'Buy'
        else: 
            dfprice['decision'] = 'Sell'

        return dfprice